In [1]:
import json
import h5py
from math import floor

In [2]:
vg_sgg = h5py.File('../data//mini_VG-SGG.h5')
imdb = h5py.File('../data/mini_imdb_1024.h5')

In [6]:
# import shutil

# shutil.rmtree('../data/train/images')
# shutil.rmtree('../data/train/labels')
# shutil.rmtree('../data/val/images')
# shutil.rmtree('../data/val/labels')

In [7]:
import os

train_image_dir = '../data/train/images'
train_label_dir = '../data/train/labels'
val_image_dir = '../data/val/images'
val_label_dir = '../data/val/labels'
os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)

In [8]:
from PIL import Image

In [9]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from math import ceil, floor

# idx = 0
# start = vg_sgg['img_to_first_box'][idx]
# end = vg_sgg['img_to_last_box'][idx]

# img = imdb['images'][idx]
# img = img.transpose(1, 2, 0)
# b = img[:][:][0].copy()
# img[:][:][0] = img[:][:][2]
# img[:][:][2] = b

# # ori_h = imdb['original_heights'][idx]
# # ori_w = imdb['original_widths'][idx]
# # w, h = 1024, 1024
# # if ori_h > ori_w:
# #   rw = ceil((1024.0/ori_h)*ori_w)
# #   w = rw
# # else:
# #   rh = ceil((1024.0/ori_w)*ori_h)
# #   h = rh
# # print(h, w)
# # ori_img = np.zeros((h, w, 3))
# # ori_img = img[:h][:w][:]
# # img = ori_img

# img = Image.fromarray(img)
# source_size = 1024
# target_size = 640

# img = img.resize((target_size, target_size))

# fig, ax = plt.subplots()

# ax.imshow(img)

# box_type = 'boxes_1024'

# end = start + min(3, end-start+1)
# for i in range(start, end):
#     x_c = (float(vg_sgg[box_type][i][0])/source_size)*target_size
#     y_c = (float(vg_sgg[box_type][i][1])/source_size)*target_size
#     w = min(1.0, float(vg_sgg[box_type][i][2])/source_size)*target_size
#     h = min(1.0, float(vg_sgg[box_type][i][3])/source_size)*target_size
#     # w = vg_sgg[box_type][i][2]
#     # h = vg_sgg[box_type][i][3]
#     # x_c = vg_sgg[box_type][i][0]
#     # y_c = vg_sgg[box_type][i][1]
#     x = ceil(x_c - (w/2))
#     y = ceil(y_c - (h/2))
#     print(x, y, w, h)
#     rect = patches.Rectangle((x,  y), w, h, linewidth=1, edgecolor='r', facecolor='none', label='hello')
#     ax.add_patch(rect)

# plt.show()


In [10]:
from tqdm.auto import tqdm

def annotate(s, e, image_dir):

  for idx in tqdm(range(s, e)):
    start = vg_sgg['img_to_first_box'][idx]
    end = vg_sgg['img_to_last_box'][idx]

    if start < 0 or end < 0:
      continue

    image_path = f'{image_dir}/images/{idx:06d}.png'
    label_path = f'{image_dir}/labels/{idx:06d}.txt'
    img = imdb['images'][idx]
    img = img.transpose(1, 2, 0)
    b = img[:][:][0].copy()
    img[:][:][0] = img[:][:][2]
    img[:][:][2] = b
    img = Image.fromarray(img)
    img.save(image_path)

    box_type = 'boxes_1024'
    source_img_size = 1024
    labels = []
    for i in range(start, end):
      label = str(vg_sgg['labels'][i][0])
      x_c = vg_sgg[box_type][i][0]/source_img_size
      y_c = vg_sgg[box_type][i][1]/source_img_size
      w = min(1, vg_sgg[box_type][i][2]/source_img_size)
      h = min(1, vg_sgg[box_type][i][3]/source_img_size)


      label = str(int(label)-1)
      label = label + f' {x_c} {y_c} {w} {h}'
      labels.append(label)

    labels = '\n'.join(labels)
    with open(label_path, 'w') as f:
      f.write(labels)

/Users/abhiram/miniconda3/envs/prpoject/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
annotate(0, 700, '../data/train')
annotate(700, 1000, '../data/val')

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 300/300 [00:59<00:00,  5.03it/s]


In [12]:
f = open('../data/mini_VG-SGG-dicts.json')
labels = json.load(f)

In [13]:
names = {}

for k in labels['idx_to_label']:
  names[int(k)-1] = labels['idx_to_label'][k]



In [14]:
import yaml

config = {
    'names': names,
    'train': '/Users/abhiram/PRProject/PRProject/data/train/images',
    'val': '/Users/abhiram/PRProject/PRProject/data/val/images'
}

with open('../data/data.yaml', 'w') as f:
    yaml.dump(config, f)

In [15]:
from ultralytics import YOLO

In [16]:
yolo = YOLO('yolov8n.pt')

In [ ]:
yolo.train(data='../data/data.yaml', epochs=10, device='mps')

In [ ]:
valid_results = yolo.val()
print(valid_results)